Autor: Ricardo Ander-Egg Aguilar

* 🖥: https://ricardoanderegg.com/
* 🐦: https://twitter.com/ricardoanderegg
* 👨🏻‍🎓: https://www.linkedin.com/in/ricardoanderegg/

In [ ]:
import requests

## Ejercicio


* Usando la API: https://exchangeratesapi.io/
    * Hay que entrar a la web y ahi sale la "documentación". Debéis usar la URL correcta para lo que queréis hacer.
* Crear una función de conversión de divisas.
* La función acepta 2 parámetros. El primer parámetro son la contidad de "EUR". El otro parámetro es la divisa a la que queremos convertir. 

**Extra**
* Añadir un tercer parámetro `lista=False`.
* Si lo ejecutamos con `lista=True` la función nos debe devolver ADEMÁS de la conversión, una lista que incluya todas las divisas entre las que puede convertir.

In [ ]:
def conversor(cantidad, divisa, lista=False):
    
    url = "https://api.exchangeratesapi.io/latest"
    res = requests.get(url)
    data = res.json()
    
    ratio = data["rates"][divisa]
    
    resultado = cantidad * ratio
    
    if lista: ## --lo mismo que--> if lista == True:
        divisas_disponibles = list(data["rates"].keys())
        
        return resultado, divisas_disponibles
    
    
    
    return resultado

Por ejemplo para convertir 25€ ---> USD:

In [ ]:
resultado, disponibles = conversor(25, "HKD", lista=True)

In [ ]:
resultado

211.9525

**Mi recomendación**

En lugar de ponerse con la función directamente, podemos aprovechar la "interactividad" que nos da jupyterlab. Podemos primero hacer la `request` y capturar el resultado. Después convertirlo a con `.json()`, ver que tipo de objeto nos devuelve, etc, etc. En resumen, ir experimentando con las variables y cuando ya estamos seguros de los pasos que se deben ejecutar, **en ese momento** cogemos lo que hemos estado haciendo y lo convertirmos en una función que acepte los parámetros adecuados.

Pero eso mejor primer ver todos los pasos que hay que hacer y como se hacen. Para eso Jupyter es perfecto. Y una vez lo tenemos ya lo pasamos a una función.

Si fuera algo más complejo incluso haríamos haciendo esto para todas las funciones y una vez lo tuviéramos todo hecho, lo pasaríamos a un script de python.

## ORM, Bases de datos con SQLAlchemy

ORM = Object relational mapper

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
# creamos una base de datos SQLite directamente en la memoria RAM del ordenador
engine = create_engine("sqlite:///:memory:")

# para crearla en el disco duro. echo=True hará que SQLAlchemy nos devuelve información del tipo de queries que está ejecutando
# engine = create_engine("sqlite:///curso.db", echo=True)

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String

Creamos nuestra tabla.


El método `__repr__` indica dentro de una clase qué queremos que aparezca cuando hagamos `print()`.

In [ ]:
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name,
            self.fullname,
            self.nickname,
        )

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
ricardo_user = User(name="ricardo", fullname="Ricardo Ander-Egg", nickname="rick")

In [ ]:
from sqlalchemy.orm import sessionmaker

In [ ]:
Session = sessionmaker(bind=engine)

In [ ]:
session = Session()

In [ ]:
session.add(ricardo_user)

Ahora podemos hacer queries sin escribir directamente SQL usando python, objetos, y la relación entre estos objetos.

In [ ]:
mi_usuario = session.query(User).filter_by(name="ricardo").first()

Añadimos varios a la vez.

In [ ]:
lista_users = [
    User(name="erlantz", fullname="erlantz saenz", nickname="er"),
    User(name="daniel", fullname="daniel orejuela", nickname="dani"),
    User(name="daniel", fullname="daniel izquierdo", nickname="dani1"),
    User(name="daniel", fullname="daniel garcia", nickname="dani2"),
    User(name="euro", fullname="euro mauro", nickname="eu"),
    
]

In [ ]:
session.add_all(lista_users)

In [ ]:
session.commit()

Si nos equivocamos podemos hacer un **`.rollback()`** antes de hacer commit!

In [ ]:
usuario_erroneo = User(name="user erroneo", fullname="oriol mitja", nickname="uri")

In [ ]:
session.add(usuario_erroneo)

Podemos hacer una query 

In [ ]:
session.query(User).filter(User.name.in_(["Manuel", "user erroneo"])).all()

[<User(name='user erroneo', fullname='oriol mitja', nickname='uri')>]

Hacemos rollback para "deshacer" la acción que habíamos ejecutado con el `.add()`

In [ ]:
session.rollback()

Para filtrar tenemos muchas opciones!

In [ ]:
for usuario in session.query(User).order_by(User.id):
    print(usuario.name, usuario.fullname, usuario.nickname)

ricardo Ricardo Ander-Egg rick
erlantz erlantz saenz er
daniel daniel orejuela dani
daniel daniel izquierdo dani1
daniel daniel garcia dani2
euro euro mauro eu


In [ ]:
for nombre, nombre_completo in session.query(User.name, User.fullname):
    print(nombre, nombre_completo)

ricardo Ricardo Ander-Egg
erlantz erlantz saenz
daniel daniel orejuela
daniel daniel izquierdo
daniel daniel garcia
euro euro mauro


El equivalente de esto en SQL sería:

```sql
SELECT users.name AS nombre,
        users.fullname AS nombre_completo
FROM users
()
```

In [ ]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

<User(name='ricardo', fullname='Ricardo Ander-Egg', nickname='rick')> ricardo
<User(name='erlantz', fullname='erlantz saenz', nickname='er')> erlantz
<User(name='daniel', fullname='daniel orejuela', nickname='dani')> daniel
<User(name='daniel', fullname='daniel izquierdo', nickname='dani1')> daniel
<User(name='daniel', fullname='daniel garcia', nickname='dani2')> daniel
<User(name='euro', fullname='euro mauro', nickname='eu')> euro


In [ ]:
for user in (
    session.query(User)
    .filter(User.name == "daniel")
    .filter(User.fullname == "daniel orejuela")
):
    print(user)

<User(name='daniel', fullname='daniel orejuela', nickname='dani')>


In [ ]:
import requests
import sqlalchemy

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

from sqlalchemy.orm import sessionmaker

## Ejercicio (se corregirá el viernes)

1. Obtener todas las cartas de la API.
1. La key `colors` contiene un value que es una lista. Filtrar las cartas que contengan `"Green"` en esa lista.
1. Generar una base de datos con SQLAlchemy para almacenar estas cartas.
1. Las columnas que debemos crear son (entre paréntesis está el nombre de la key que tienen en el diccionario):
    * nombre (`name`)
    * multiverse_id (`multiverseid`)  <== queda eliminado a menos que queraís hacer la parte extra
    * url_imagen (`imageUrl`) || tipo (`type`)
    * rareza (`rarity`)
    * **Extra**: en las cartas verdes que NO tienen `multiverseid`, crearlo y darle el valor `0`
    
1. De la lista de cartas filtradas que hemos obtenido en el punto $2$, guardarlas todas en la base de datos.

In [ ]:
r = requests.get("https://api.magicthegathering.io/v1/cards")

data = r.json()
cartas = data["cards"]

verdes = []

for carta in cartas:
    
    colores = carta["colors"]
    
    if "Green" in colores:
        
        ##### CODE #####
        pass

In [ ]:
engine = create_engine("sqlite:///:memory:", echo=True)

Base = declarative_base()

Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)

session = Session()

2020-04-02 14:44:41,468 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-04-02 14:44:41,474 INFO sqlalchemy.engine.base.Engine ()
2020-04-02 14:44:41,479 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-04-02 14:44:41,483 INFO sqlalchemy.engine.base.Engine ()


In [ ]:
class Verde(Base):
    __tablename__ = "verdes"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    # multiverse_id = Column(Integer)
    url = Column(String)
    rarity = Column(String)

    def __repr__(self):
        return f"<User(name='{self.name}', url='{self.url}', rarity='{self.rarity}')>"

In [ ]:
nueva_carta = Verde(######## CODE ######)
session.add(nueva_carta)

In [ ]:
lista_users = [
    User(name="erlantz", fullname="erlantz saenz", nickname="er"),
    User(name="daniel", fullname="daniel orejuela", nickname="dani"),
    User(name="daniel", fullname="daniel izquierdo", nickname="dani1"),
]

In [ ]:
session.add_all(lista_users)

In [ ]:
session.commit()

In [ ]:
for user in (
    session.query(User)
    .filter(User.name == "daniel")
    .filter(User.fullname == "daniel orejuela")
):
    print(user)

In [ ]:
for nombre, nombre_completo in session.query(User.name, User.fullname):
    print(nombre, nombre_completo)

**Extra 🔥**

* Crear una función que obtenga la información de la carta de la base de datos en base a un `id` o como queráis.
* Descargar la imagen de esta carta (o de la cabra 😂) y guardarla en un archivo en el disco.

Por ejemplo:

```python
req_img = requests.get(v[0]["imageUrl"])

with open("imagen.jpg", "wb") as f:
    f.write(req_img.content)
```

## Comprehensions

Explicación animada en formato GIF:

Fuente del GIF y lectura recomendada: https://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/

![](https://treyhunner.com/images/list-comprehension-condition.gif)

In [ ]:
import requests

In [ ]:
r = requests.get("https://api.magicthegathering.io/v1/cards")

data = r.json()

baraja = data["cards"]

Nos permiten convertir esto:

In [ ]:
lista = []

for carta in baraja:
    lista.append(carta["name"])

En esto:

In [ ]:
lista_nombres = [carta["name"] for carta in baraja]

Podemos usar condiciones, incluso más de una.

In [ ]:
verdes_azules = [
    carta for carta in baraja if "Green" in carta["colors"] or "Blue" in carta["colors"]
]

Para procesar textos.

Descargamos una lista de "stop words" (palabras que carecen de un significado).

In [ ]:
req_stop = requests.get("https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt")

Obtenemos el texto de esa request.

In [ ]:
string_stop = req_stop.text

Lo convertimos en lista usando `.split()`

In [ ]:
lista_stop = string_stop.split("\n")

Vamos a trabajar con esta cadena de texto.

In [ ]:
texto_raw = """
Python 2.0 tomó una característica mayor del lenguaje de programación funcional Haskell: listas por comprensión. La sintaxis de Python para esta construcción es muy similar a la de Haskell, salvo por la preferencia de los caracteres de puntuación en Haskell, y la preferencia de Python por palabras claves alfabéticas. Python 2.0 introdujo además un sistema de recolección de basura capaz de recolectar referencias cíclicas.13
En 1991, van Rossum publicó el código de la versión 0.9.0 en alt.sources.9​ En esta etapa del desarrollo ya estaban presentes clases con herencia, manejo de excepciones, funciones y los tipos modulares, como: str, list, dict, entre otros. Además en este lanzamiento inicial aparecía un sistema de módulos adoptado de Modula-3; van Rossum describe el módulo como «una de las mayores unidades de programación de Python».4​ El modelo de excepciones en Python es parecido al de Modula-3, con la adición de una cláusula else.5​ En el año 1994 se formó comp.lang.python, el foro de discusión principal de Python, marcando un hito en el crecimiento del grupo de usuarios de este lenguaje.

Python alcanzó la versión 1.0 en enero de 1994. Una característica de este lanzamiento fueron las herramientas de la programación funcional: lambda, reduce, filter y map. Van Rossum explicó que «hace 12 años, Python adquirió lambda, reduce(), filter() y map(), cortesía de un hacker informático de Lisp que las extrañaba y que envió parches».10​ El donante fue Amrit Prem; no se hace ninguna mención específica de cualquier herencia de Lisp en las notas de lanzamiento.

La última versión liberada proveniente de CWI fue Python 1.2. En 1995, van Rossum continuó su trabajo en Python en la Corporation for National Research Initiatives (CNRI) en Reston, Virginia, donde lanzó varias versiones del software.

Durante su estancia en CNRI, van Rossum lanzó la iniciativa Computer Programming for Everybody (CP4E), con el fin de hacer la programación más accesible a más gente, con un nivel de 'alfabetización' básico en lenguajes de programación, similar a la alfabetización básica en inglés y habilidades matemáticas necesarias por muchos trabajadores. Python tuvo un papel crucial en este proceso: debido a su orientación hacia una sintaxis limpia, ya era idóneo, y las metas de CP4E presentaban similitudes con su predecesor, ABC. El proyecto fue patrocinado por DARPA.11​ En el año 2007, el proyecto CP4E está inactivo, y mientras Python intenta ser fácil de aprender y no muy arcano en su sintaxis y semántica, alcanzando a los no-programadores, no es una preocupación activa.12​

En el año 2000, el equipo principal de desarrolladores de Python se cambió a BeOpen.com para formar el equipo BeOpen PythonLabs. CNRI pidió que la versión 1.6 fuera pública, continuando su desarrollo hasta que el equipo de desarrollo abandonó CNRI; su programa de lanzamiento y el de la versión 2.0 tenían una significativa cantidad de traslapo.13​ Python 2.0 fue el primer y único lanzamiento de BeOpen.com. Después que Python 2.0 fuera publicado por BeOpen.com, Guido van Rossum y los otros desarrolladores de PythonLabs se unieron en Digital Creations.

Python 2.0 tomó una característica mayor del lenguaje de programación funcional Haskell: listas por comprensión. La sintaxis de Python para esta construcción es muy similar a la de Haskell, salvo por la preferencia de los caracteres de puntuación en Haskell, y la preferencia de Python por palabras claves alfabéticas. Python 2.0 introdujo además un sistema de recolección de basura capaz de recolectar referencias cíclicas.13​

Posterior a este doble lanzamiento, y después que van Rossum dejó CNRI para trabajar con desarrolladores de software comercial, quedó claro que la opción de usar Python con software disponible bajo GNU GPL era muy deseable. La licencia usada entonces, la Python License, incluía una cláusula estipulando que la licencia estaba gobernada por el estado de Virginia, por lo que, bajo la óptica de los abogados de Free Software Foundation (FSF), se hacía incompatible con GPL. CNRI y FSF se relacionaron para cambiar la licencia de software libre de Python para hacerla compatible con GPL. En el año 2001, van Rossum fue premiado con FSF Award for the Advancement of Free Software.
En 1991, van Rossum publicó el código de la versión 0.9.0 en alt.sources.9​ En esta etapa del desarrollo ya estaban presentes clases con herencia, manejo de excepciones, funciones y los tipos modulares, como: str, list, dict, entre otros. Además en este lanzamiento inicial aparecía un sistema de módulos adoptado de Modula-3; van Rossum describe el módulo como «una de las mayores unidades de programación de Python».4​ El modelo de excepciones en Python es parecido al de Modula-3, con la adición de una cláusula else.5​ En el año 1994 se formó comp.lang.python, el foro de discusión principal de Python, marcando un hito en el crecimiento del grupo de usuarios de este lenguaje.

Python alcanzó la versión 1.0 en enero de 1994. Una característica de este lanzamiento fueron las herramientas de la programación funcional: lambda, reduce, filter y map. Van Rossum explicó que «hace 12 años, Python adquirió lambda, reduce(), filter() y map(), cortesía de un hacker informático de Lisp que las extrañaba y que envió parches».10​ El donante fue Amrit Prem; no se hace ninguna mención específica de cualquier herencia de Lisp en las notas de lanzamiento.

La última versión liberada proveniente de CWI fue Python 1.2. En 1995, van Rossum continuó su trabajo en Python en la Corporation for National Research Initiatives (CNRI) en Reston, Virginia, donde lanzó varias versiones del software.

Durante su estancia en CNRI, van Rossum lanzó la iniciativa Computer Programming for Everybody (CP4E), con el fin de hacer la programación más accesible a más gente, con un nivel de 'alfabetización' básico en lenguajes de programación, similar a la alfabetización básica en inglés y habilidades matemáticas necesarias por muchos trabajadores. Python tuvo un papel crucial en este proceso: debido a su orientación hacia una sintaxis limpia, ya era idóneo, y las metas de CP4E presentaban similitudes con su predecesor, ABC. El proyecto fue patrocinado por DARPA.11​ En el año 2007, el proyecto CP4E está inactivo, y mientras Python intenta ser fácil de aprender y no muy arcano en su sintaxis y semántica, alcanzando a los no-programadores, no es una preocupación activa.12​

En el año 2000, el equipo principal de desarrolladores de Python se cambió a BeOpen.com para formar el equipo BeOpen PythonLabs. CNRI pidió que la versión 1.6 fuera pública, continuando su desarrollo hasta que el equipo de desarrollo abandonó CNRI; su programa de lanzamiento y el de la versión 2.0 tenían una significativa cantidad de traslapo.13​ Python 2.0 fue el primer y único lanzamiento de BeOpen.com. Después que Python 2.0 fuera publicado por BeOpen.com, Guido van Rossum y los otros desarrolladores de PythonLabs se unieron en Digital Creations.

Python 2.0 tomó una característica mayor del lenguaje de programación funcional Haskell: listas por comprensión. La sintaxis de Python para esta construcción es muy similar a la de Haskell, salvo por la preferencia de los caracteres de puntuación en Haskell, y la preferencia de Python por palabras claves alfabéticas. Python 2.0 introdujo además un sistema de recolección de basura capaz de recolectar referencias cíclicas.13​

Posterior a este doble lanzamiento, y después que van Rossum dejó CNRI para trabajar con desarrolladores de software comercial, quedó claro que la opción de usar Python con software disponible bajo GNU GPL era muy deseable. La licencia usada entonces, la Python License, incluía una cláusula estipulando que la licencia estaba gobernada por el estado de Virginia, por lo que, bajo la óptica de los abogados de Free Software Foundation (FSF), se hacía incompatible con GPL. CNRI y FSF se relacionaron para cambiar la licencia de software libre de Python para hacerla compatible con GPL. En el año 2001, van Rossum fue premiado con FSF Award for the Advancement of Free Software.
Python 2.1 fue un trabajo derivado de Python 1.6.1, así como también de Python 2.0. Su licencia fue renombrada a: Python Software Foundation License. Todo el código, documentación y especificaciones añadidas, desde la fecha del lanzamiento de la versión alfa de Python 2.1, tiene como dueño a Python Software Foundation (PSF), una organización sin ánimo de lucro fundada en el año 2001, tomando como modelo la Apache Software Foundation.3​ Incluido en este lanzamiento fue una implementación del scoping más parecida a las reglas de static scoping (del cual Scheme es el originador).14​

Una innovación mayor en Python 2.2 fue la unificación de los tipos en Python (tipos escritos en C), y clases (tipos escritos en Python) dentro de una jerarquía. Esa unificación logró un modelo de objetos de Python puro y consistente.15​ También fueron agregados los generadores que fueron inspirados por el lenguaje Icon.16​

Las adiciones a la biblioteca estándar de Python y las decisiones sintácticas fueron influenciadas fuertemente por Java en algunos casos: el package logging,17​ introducido en la versión 2.3, está basado en log4j; el parser SAX, introducido en 2.0; el package threading,18​ cuya clase Thread expone un subconjunto de la interfaz de la clase homónima en Java.

Python 2, es decir Python 2.7.x, fue oficialmente descontinuado el 1 de enero de 2020 (primero planeado para 2015) después de lo cual no se publicarán parches de seguridad y otras mejoras para él.19​20​ Con el final del ciclo de vida de Python 2, solo tienen soporte la rama Python 3.5.x21​ y posteriores.
Python 1.6.1 es esencialmente el mismo que Python 1.6, con unos pocos arreglos de bugs, y con una nueva licencia compatible con GPL.3​
Python 1.6.1 es esencialmente el mismo que Python 1.6, con unos pocos arreglos de bugs, y con una nueva licencia compatible con GPL.3​
"""

Lo convertimos en minúsculas y lo dividimos por palabras.

In [ ]:
texto_lista = texto_raw.lower().split()

Con la siguiente comprensión de lista creamos una lista nueva pero donde hemos eliminado las palabras que estaban en la lista `stop_words`. Lo hacemos usando el condicional:

```python
if palabra not in lista_stop
```
que está al final.

In [ ]:
texto_procesado = [palabra for palabra in texto_lista if palabra not in lista_stop]

El equivalente en un for loop sería:

In [ ]:
texto_procesado_2 = []

for p in texto_lista:
    
    if p not in lista_stop:
        texto_procesado_2.append(p)

También podemos construir diccionarios! Y añadiendo condiciones por ejemplo:

```python
palabra.startswith("p")
```

Y seleccionaremos solamente las palabras que empiezan por "p".

In [ ]:
{palabra: ("i" in palabra) for palabra in texto_procesado if palabra.startswith("p")}

{'python': False,
 'programación': True,
 'preferencia': True,
 'puntuación': True,
 'palabras': False,
 'publicó': True,
 'presentes': False,
 'python».4\u200b': False,
 'parecido': True,
 'principal': True,
 'python,': False,
 'parches».10\u200b': False,
 'prem;': False,
 'proveniente': True,
 'programming': True,
 'programación,': True,
 'papel': False,
 'proceso:': False,
 'presentaban': False,
 'predecesor,': False,
 'proyecto': False,
 'patrocinado': True,
 'preocupación': True,
 'pythonlabs.': False,
 'pidió': True,
 'pública,': True,
 'programa': False,
 'primer': True,
 'publicado': True,
 'pythonlabs': False,
 'posterior': True,
 'premiado': True,
 'parecida': True,
 'python)': False,
 'puro': False,
 'package': False,
 'parser': False,
 'planeado': False,
 'publicarán': True,
 'parches': False,
 'posteriores.': True,
 'pocos': False}